<a href="https://colab.research.google.com/github/saeidsaadatigero/AI_ML_Computer-Vision/blob/main/22_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ========================
# 📘 Logistic Regression با PyTorch روی MNIST (دودویی 0 یا 1)
# ========================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# --- تنظیمات پایه ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- پیش‌پردازش داده‌ها ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# --- دانلود دیتاست MNIST ---
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# فقط نمونه‌های عدد 0 و 1 را نگه داریم تا مسئله دوکلاسه شود
train_idx = [i for i, t in enumerate(train_data.targets) if t in [0, 1]]
test_idx = [i for i, t in enumerate(test_data.targets) if t in [0, 1]]

train_subset = Subset(train_data, train_idx)
test_subset = Subset(test_data, test_idx)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

# --- تعریف مدل ساده Logistic Regression ---
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28*28, 1)  # 784 ورودی → 1 خروجی (احتمال)

    def forward(self, x):
        x = x.view(-1, 28*28)   # تبدیل تصویر به وکتور
        return torch.sigmoid(self.linear(x))  # خروجی بین 0 و 1

model = LogisticRegressionModel().to(device)

# --- تابع هزینه و بهینه‌ساز ---
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.1)

# --- آموزش مدل ---
for epoch in range(5):
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"دوره {epoch+1} - میانگین خطا: {total_loss/len(train_loader):.4f}")

# --- ارزیابی روی داده تست ---
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"\n✅ دقت مدل روی داده تست: {100 * correct / total:.2f}%")

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.53MB/s]


دوره 1 - میانگین خطا: 0.0462
دوره 2 - میانگین خطا: 0.0058
دوره 3 - میانگین خطا: 0.0044
دوره 4 - میانگین خطا: 0.0036
دوره 5 - میانگین خطا: 0.0036

✅ دقت مدل روی داده تست: 99.91%
